# Carnival Demo - GPTWaterLevelModel

In [1]:
!rm -rf GPTWaterLevelModel
!git clone https://github.com/ToletiSri/GPTWaterLevelModel.git


Cloning into 'GPTWaterLevelModel'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 33 (delta 11), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (33/33), 90.07 KiB | 572.00 KiB/s, done.


In [2]:
cd GPTWaterLevelModel

/kaggle/working/GPTWaterLevelModel


### Create and train the model

In [3]:
import pandas as pd
import torch
import config as cfg
from gpt import GPTWaterLevelModel

In [4]:
# Read the values from excel
df = pd.read_csv('teraterm_sensor.csv', sep=',', header=None)
sensor_data = torch.tensor(df.values)
print(sensor_data.shape, sensor_data.dtype)
print(sensor_data[1:10, :])

torch.Size([326, 3]) torch.float64
tensor([[48.8853,  2.2496,  7.2400],
        [48.8853,  2.2496,  7.2300],
        [48.8853,  2.2496,  7.2200],
        [48.8853,  2.2496,  7.2100],
        [48.8853,  2.2496,  7.2000],
        [48.8853,  2.2496,  7.1900],
        [48.8853,  2.2496,  7.1800],
        [48.8853,  2.2496,  7.1700],
        [48.8853,  2.2496,  7.1600]], dtype=torch.float64)


In [5]:
# Train and test splits
n = int(1.0*len(sensor_data)) # Take 100%, currently we dont have sufficient data for train and test
train_data = sensor_data[:n]
val_data = sensor_data[n:]
print(train_data.shape, train_data.dtype)
print(val_data.shape, val_data.dtype)

torch.Size([326, 3]) torch.float64
torch.Size([0, 3]) torch.float64


In [6]:
# Read parameters from configuration
device = cfg.device
block_size = cfg.block_size
batch_size = cfg.batch_size
eval_iters = cfg.eval_iters
print(device)

cuda


In [7]:
# data loading
def get_batch(split, batch_size = 1):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.float().to(device), y.float().to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train']: #, 'val'
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size)
            Y = Y[:, :, 2] # Pass only water level as expected output
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
# Model Training
from gpt import GPTWaterLevelModel
model = GPTWaterLevelModel(800).to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)

for iter in range(cfg.max_iters):

    #every once in a while evaluate the loss on train and val sets
    if iter % cfg.eval_interval == 0 or iter == cfg.max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: loss {losses['train']:.4f}")    # , val loss {losses['val']:.4f}     

    # sample a batch of data
    xb, yb = get_batch('train', batch_size)
    yb = yb[:, :, 2] # Pass only water level as expected output

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
torch.save(model.state_dict(), 'saved_model.pth')

5.121056 M parameters
step 0: loss 6.7323
step 100: loss 5.8139
step 200: loss 5.7792
step 300: loss 4.0265
step 400: loss 0.2870
step 500: loss 0.0537
step 600: loss 0.0261
step 700: loss 0.0168
step 800: loss 0.0117
step 900: loss 0.0088
step 999: loss 0.0070


### Generate the next few water levels based on current water levels

In [9]:
# sample a batch of data
xb, _ = get_batch('train',1)
# Take the first 8 samples from the batch and generate next 8 values
CurrentWaterLevels = xb[:,:8,:] # Take the first 8 samples. Data = latitude, longitude, water_level of first 8 elements
next_coordinates = xb[:,8:12,0:2] # Take the co-ordinates of next 8 samples. Data = latitude, longitude of next 8 elements
# Predict the next 8 water levels at the given next 8 co-ordinates
predicted_water_levels = model.generate(CurrentWaterLevels, 4, next_coordinates) 
print(f'Current 8 water levels = {[round(value, 4) for value in CurrentWaterLevels[:,:,-1].tolist()[0]]}' )
print(f'Actual next 4 water levels = {[round(value, 4) for value in xb[:,8:12,-1].tolist()[0]]}' )
print(f'Predicted next 4 water levels = {[round(value,4) for value in predicted_water_levels[:,8:,-1].tolist()[0]]}' )

Current 8 water levels = [6.14, 6.13, 6.12, 6.11, 6.1, 6.09, 6.08, 6.07]
Actual next 4 water levels = [6.06, 6.05, 6.04, 6.03]
Predicted next 4 water levels = [6.06, 6.05, 6.04, 6.03]
